In [1]:
from src.data.data_retriever import DataRetriever
from src.data.odoo_connector import OdooConnection
import asyncio
from config.settings import settings
import nest_asyncio
import pandas as pd
import numpy as np
nest_asyncio.apply()

# Exploración de los datos

## 1. Estructura de la base de datos

### 1.1. Modelos relevantes

En está sección describo los distintos modelos que deberán ser utilizados. 

También he seleccionado los campos que pueden llegar a ser importantes para el desarrollo. Cabe remarcar que con "importantes", no me refiero únicamente para entrenar el modelo de predicción de impagos, también tengo en cuenta información que el agente podría necesitar.

Más adelante analizo la estructura de cada modelo y campo y comento sobre los datos que serán útiles o no para el entrenamiento

#### 1.1.1. res.company

Contiene información sobre las empresas que forman el grupo (no clientes):
- id
- name
- currency_id (Identificador de la moneda [id, nombre])

#### 1.1.2. res.partner

Contiene información sobre los partners (clientes/proveedores):
- id
- name
- email
- phone
- street
- city
- zip
- country_id
- customer_rank (>0 es cliente)
- supplier_rank (>0 es proveedor)
- category_id (sector/industria a la que pertenece)
- is_company
- company_type
- credit
- credit_limit
- debit
- debit_limit
- industry_id
- invoice_ids
- total_due
- total_invoiced
- total_overdue
- trust
- unpaid_invoice_ids
- unpaid_invoices_count

#### 1.1.3. account.move

Guarda todas las facturas y movimientos contables de la empresa, es decir, los registros de todo lo que se compra, se vende o se paga. Este será el modelo principal con el que trabajará el agente:
- id
- name
- move_type ("out_invoice", "in_invoice", "out_refund", "in_refund", "entry")
- payment_state ("not_paid", "in_payment", "paid", "partial", "reversed")
- company_id
- partner_id
- currency_id
- amount_total
- amount_paid
- amount_residual
- invoice_date
- invoice_date_due
- payment_dates
- date
- create_date
- payment_id
- payment_ids

#### 1.1.4. res.currency

Contiene información sobre las monedas en las que se emiten facturas y se registran los movimientos:
- id
- name
- symbol
- rate

#### 1.1.5. res.country

Contiene información sobre los países:
- id
- name
- code

#### 1.1.6. res.partner.category

Representa las categorías asignadas a los partners:
- id
- name

#### 1.1.7. res.partner.industry

Contiene información de la industria / sector económico de los partners:
- id
- name

### 1.2. Características de los datos

#### Conexión a Odoo

In [3]:
odoo_connection = OdooConnection()
asyncio.run(odoo_connection.connect())
data_retriever = DataRetriever(odoo_connection=odoo_connection)

Connected to Odoo as albert.gil@yourtechtribe.com (uid: 430)
Odoo server version: {'server_version': '16.0+e-20250313', 'server_version_info': [16, 0, 0, 'final', 0, 'e'], 'server_serie': '16.0', 'protocol_version': 1}


#### 1.2.1. res.company

In [4]:
company_data = asyncio.run(data_retriever.get_all_companies())
company_df = pd.DataFrame([c.to_dict() for c in company_data])
company_df

,id,name,currency_id
0,1,"Grupo Viko Digital Marketing, S.A.","(1, EUR)"
1,3,Elogia Media S.L.,"(1, EUR)"
2,2,Ibrands Medios Interactivos SL,"(1, EUR)"
3,5,Kraz Data Solutions SL,"(1, EUR)"
4,6,Marketing4ecommerce Digital Content SL,"(1, EUR)"
5,13,Octoplus Digital Shelf Optimization SL,"(1, EUR)"
6,7,Tandem Trade Marketing SL,"(1, EUR)"
7,11,"DigitalPla2021, S.L.","(1, EUR)"
8,14,Ideas y Estrategia Digital SL,"(1, EUR)"
9,8,INICIATIVAS VIRTUALES DE MEXICO,"(33, MXN)"


El grupo está formado por 12 empresas, 9 usan el euro y 3 el peso mexicano.

#### 1.2.2. res.partner

In [5]:
partners_data = asyncio.run(data_retriever.get_all_partners())
partners_df = pd.DataFrame([p.to_dict() for p in partners_data])
partners_df

Recuperadas 500 facturas, total: 500
Recuperadas 500 facturas, total: 1000


KeyboardInterrupt: 

In [ ]:
!pip install pydantic[email]

In [ ]:
pip install mcp_odoo/.

In [7]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


Connected to Odoo as albert.gil@yourtechtribe.com (uid: 430)
Odoo server version: {'server_version': '16.0+e-20250313', 'server_version_info': [16, 0, 0, 'final', 0, 'e'], 'server_serie': '16.0', 'protocol_version': 1}


# Datos empresas

In [15]:
asyncio.run(data_retriever.get_all_companies())

[Company(id=1, name='Grupo Viko Digital Marketing, S.A.', currency_id=(1, 'EUR')),
 Company(id=3, name='Elogia Media S.L.', currency_id=(1, 'EUR')),
 Company(id=2, name='Ibrands Medios Interactivos SL', currency_id=(1, 'EUR')),
 Company(id=5, name='Kraz Data Solutions SL', currency_id=(1, 'EUR')),
 Company(id=6, name='Marketing4ecommerce Digital Content SL', currency_id=(1, 'EUR')),
 Company(id=13, name='Octoplus Digital Shelf Optimization SL', currency_id=(1, 'EUR')),
 Company(id=7, name='Tandem Trade Marketing SL', currency_id=(1, 'EUR')),
 Company(id=11, name='DigitalPla2021, S.L.', currency_id=(1, 'EUR')),
 Company(id=14, name='Ideas y Estrategia Digital SL', currency_id=(1, 'EUR')),
 Company(id=8, name='INICIATIVAS VIRTUALES DE MEXICO', currency_id=(33, 'MXN')),
 Company(id=12, name='IBRANDS MEDIOS INTERACTIVOS DE MEXICO', currency_id=(33, 'MXN')),
 Company(id=9, name='MITTUM MARKETING RELACIONAL', currency_id=(33, 'MXN'))]

In [12]:
fields = [
        "id",
        "name",
        "email",
        "phone",
        "street",
        "city",
        "zip",
        "country_id",
        "customer_rank",
        "supplier_rank",
        "category_id",
        "company_type",
        "credit",
        "credit_limit",
        "debit",
        "debit_limit",
        "industry_id",
        "invoice_ids",
        "total_due",
        "total_invoiced",
        "total_overdue",
        "trust",
        "unpaid_invoice_ids",
        "unpaid_invoices_count",
    ]

partners = await odoo_connection.search_read(
        model="res.partner",
        domain=[("customer_rank", ">", 0)],
        fields=fields,
        limit=1000,
        offset=0
    )

In [24]:
payments = await odoo_connection.search_read(
        model="res.partner",
        domain=[("payment_type", "=", "inbound")],
        fields=[],
        limit=1000,
        offset=0
    )

OdooConnectionError: Error executing search_read on res.partner: <Fault 1: 'Traceback (most recent call last):\n  File "/opt/odoo/odoo/odoo/addons/base/controllers/rpc.py", line 151, in xmlrpc_2\n    response = self._xmlrpc(service)\n               ^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/addons/base/controllers/rpc.py", line 127, in _xmlrpc\n    result = dispatch_rpc(service, method, params)\n             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/http.py", line 369, in dispatch_rpc\n    return dispatch(method, params)\n           ^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/service/model.py", line 56, in dispatch\n    res = execute_kw(db, uid, *params[3:])\n          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/service/model.py", line 79, in execute_kw\n    return execute(db, uid, obj, method, *args, **kw or {})\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/service/model.py", line 

In [20]:
count_inbound = await odoo_connection.execute_kw(
    model="account.payment",
    method="search_count",
    args=[[("payment_type", "=", "outbound")]]
)

In [21]:
count_inbound

6592

In [15]:
for p in partners[:100]:
    print(p)

{'id': 14516, 'name': '200 Labs Inc', 'email': False, 'phone': False, 'street': '490 Post St, Ste 526', 'city': 'San Francisco', 'zip': '94102', 'country_id': [233, 'United States'], 'customer_rank': 1, 'supplier_rank': 5, 'category_id': [], 'company_type': 'company', 'credit': 0.0, 'credit_limit': 0.0, 'debit': 600.0, 'debit_limit': 0.0, 'industry_id': False, 'invoice_ids': [196646, 196260, 197176, 196645, 196263, 196642, 196262, 196259, 173406, 164674], 'total_due': 0.0, 'total_invoiced': 1808.5, 'total_overdue': 0.0, 'trust': 'normal', 'unpaid_invoice_ids': [], 'unpaid_invoices_count': 0}
{'id': 12500, 'name': '202 Digital Reputation, S.L.', 'email': False, 'phone': False, 'street': 'C/ Tuset 19, entresuelo', 'city': 'Barcelona', 'zip': '08006', 'country_id': [68, 'Spain'], 'customer_rank': 3, 'supplier_rank': 0, 'category_id': [], 'company_type': 'company', 'credit': 0.0, 'credit_limit': 0.0, 'debit': 0.0, 'debit_limit': 0.0, 'industry_id': False, 'invoice_ids': [101525, 100842, 10

# Datos facturas

In [32]:
company_id = 14
invoices = asyncio.run(data_retriever.get_all_outbound_invoices(company_id))
invoices_df = pd.DataFrame([i.to_dict() for i in invoices])

Recuperadas 500 facturas, total: 500
Recuperadas 95 facturas, total: 595


In [33]:
invoices_df.describe()

,id,amount_total,amount_residual,days_overdue
count,595.000000,595.000000,595.000000,595.000000
mean,143420.842017,5338.248672,912.600571,7.858824
std,21899.350306,8546.715734,4151.656529,13.038414
min,121169.000000,108.730000,0.000000,-1.000000
25%,130328.500000,955.900000,0.000000,0.000000
50%,130477.000000,2178.000000,0.000000,4.000000
75%,157948.500000,6198.225000,0.000000,11.500000
max,194734.000000,90096.600000,41964.870000,103.000000


In [34]:
invoices_df.columns.values

array(['id', 'name', 'move_type', 'payment_state', 'company_id',
       'partner_id', 'currency_id', 'amount_total', 'amount_residual',
       'invoice_date', 'invoice_date_due', 'journal_id', 'payment_dates',
       'paid_late', 'days_overdue'], dtype=object)

In [35]:
invoices_df["payment_dates"].dtypes

dtype('O')

In [36]:
invoices_df["payment_state"].value_counts()

payment_state
paid        515
not_paid     80
Name: count, dtype: int64

In [37]:
invoices_df["paid_late"].value_counts()

paid_late
True     365
False    150
Name: count, dtype: int64

In [38]:
invoices_df["partner_id"].value_counts()

partner_id
(13557, Chiesi España, S.A)                          124
(13548, Esteve Pharmaceuticals SA)                   113
(13558, Elanco Spain S.L.)                           106
(8933, Zambon, S.A.U.)                                66
(13559, Kern Pharma S.L.)                             47
(13549, Swedish Orphan Biovitrum S.L.)                34
(13542, Laboratorio Reig Jofre, S.A.)                 21
(13547, Alexion Pharma Nordics AB)                    18
(13543, Alexion Pharma GMBH)                          11
(14595, Swedish Orphan Biovitrum AB (publ))            8
(7583, Reckitt Benckiser Healthcare, S.A.)             8
(14476, Faes Farma S.A.)                               7
(13654, Atika Pharma S.L.)                             7
(13545, Elanco AH Portugal Unipessoal Lda.)            6
(13544, Angelini Pharma Portugal, Unipessoal Lda)      3
(13553, AdSalutem Lullaai S.L)                         2
(14627, Asociación Española Contra el Cáncer)          2
(10, Elogia Media S.

In [16]:
invoices_df["currency_id"].value_counts()

currency_id
(1, EUR)      12388
(18, SEK)         9
(142, GBP)        1
Name: count, dtype: int64

In [10]:
invoices_df

,id,name,move_type,payment_state,company_id,partner_id,currency_id,amount_total,amount_residual,invoice_date,invoice_date_due,journal_id,payment_dates,paid_late,days_overdue
0,196743,FVE/2025/00376,out_invoice,not_paid,"(3, Elogia Media S.L.)","(8920, Boehringer Ingelheim España, S.A.)","(1, EUR)",242.00,242.00,2025-06-17,2025-08-16,"(28, Facturas ventas Elogia)",None,None,-1
1,196654,FVE/2025/00375,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",453.75,453.75,2025-06-12,2025-08-11,"(28, Facturas ventas Elogia)",None,None,-1
2,196649,FVE/2025/00374,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",907.50,907.50,2025-06-12,2025-08-11,"(28, Facturas ventas Elogia)",None,None,-1
3,196631,FVE/2025/00373,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",1210.00,1210.00,2025-06-11,2025-08-10,"(28, Facturas ventas Elogia)",None,None,-1
4,196628,FVE/2025/00372,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",1210.00,1210.00,2025-06-11,2025-08-10,"(28, Facturas ventas Elogia)",None,None,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,9691,INV1/2022/00005,out_invoice,paid,"(3, Elogia Media S.L.)","(9247, Apoteca Natura Spa)","(1, EUR)",2250.00,0.00,2022-12-31,2023-01-30,"(340, Carga facturas venta Elogia)",2023-02-01,True,2
6344,9690,INV1/2022/00004,out_invoice,paid,"(3, Elogia Media S.L.)","(8301, Goiko Grill Group SL.)","(1, EUR)",11555.50,0.00,2022-12-31,2023-01-29,"(340, Carga facturas venta Elogia)",2023-02-17,True,19
6345,9689,INV1/2022/00003,out_invoice,paid,"(3, Elogia Media S.L.)","(7439, Boehringer Ingelheim Animal Health Espa...","(1, EUR)",1597.20,0.00,2022-12-31,2023-02-05,"(340, Carga facturas venta Elogia)",2023-03-01,True,24
6346,9688,INV1/2022/00002,out_invoice,paid,"(3, Elogia Media S.L.)","(8663, Irmaos Vila Nova SA)","(1, EUR)",12195.41,0.00,2022-12-31,2023-02-18,"(340, Carga facturas venta Elogia)",2023-02-21,True,3
